# BASELINE MODEL
-----------------
-----------------
-----------------

In [1]:
import pandas as pd
import numpy as np
import time
import gc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, GlobalMaxPooling2D, LSTM, MaxPooling2D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

In [2]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       6.9Gi       4.0Gi       229Mi       4.7Gi       8.1Gi
Swap:          976Mi        45Mi       931Mi


In [3]:
# This df will be a row-concatenated version of the same channels
# baseline = pd.read_csv('../../baseline.csv', delimiter=',')
# baseline.drop('Unnamed: 0', inplace=True, axis=1)

In [3]:
df = pd.read_csv('../data/dfTensor.csv', delimiter=',')
df.drop('Unnamed: 0', inplace=True, axis=1)

In [4]:
df.head()

,O1,O2,F3,F4,F7,F8,T7,T8,AF3,AF4,FC5,FC6,P7,P8,code
0,"[4203.076923, 4193.333333, 4194.871794, 4207.1...","[4229.743589, 4216.923076, 4207.179487, 4215.3...","[4538.461538, 4528.717948, 4524.615384, 4526.1...","[4682.051282, 4667.179487, 4662.051282, 4669.2...","[4489.230769, 4475.384615, 4474.358974, 4486.6...","[3989.23077, 3983.589744, 3987.692308, 3991.79...","[4497.948717, 4498.461538, 4494.871794, 4497.9...","[4506.666666, 4501.025641, 4496.923076, 4496.9...","[4395.384615, 4382.564102, 4377.435897, 4387.1...","[4078.461538, 4062.564102, 4055.897435, 4065.6...","[4207.692307, 4205.641025, 4200.51282, 4194.35...","[4227.692307, 4215.384615, 4210.76923, 4221.02...","[4203.076923, 4192.820512, 4194.871794, 4204.6...","[4245.641025, 4236.410256, 4218.461538, 4220.0...",6
1,"[4396.410256, 4388.717948, 4390.76923, 4395.89...","[4208.717948, 4197.435897, 4198.461538, 4203.0...","[4522.564102, 4522.051282, 4527.179487, 4521.0...","[4633.846153, 4619.487179, 4617.948717, 4618.9...","[4482.564102, 4477.435897, 4484.102564, 4484.6...","[4016.923076, 3996.410257, 3991.282052, 3996.4...","[4498.974358, 4500.0, 4499.487179, 4494.871794...","[4507.692307, 4492.307692, 4484.102564, 4492.8...","[4385.641025, 4381.538461, 4386.666666, 4383.5...","[4061.025641, 4044.615384, 4051.282051, 4060.5...","[4217.435897, 4221.538461, 4228.717948, 4224.6...","[4213.333333, 4189.743589, 4191.794871, 4211.7...","[4194.358974, 4192.307692, 4191.794871, 4188.2...","[4228.205128, 4214.871794, 4211.282051, 4213.8...",7
2,"[4438.974358, 4440.51282, 4445.128205, 4445.12...","[4219.487179, 4224.102564, 4230.25641, 4231.28...","[4534.871794, 4545.641025, 4552.820512, 4551.2...","[4624.615384, 4634.871794, 4646.666666, 4651.2...","[4468.717948, 4480.51282, 4493.333333, 4490.25...","[3982.564103, 3997.435898, 4006.666666, 4002.5...","[4488.717948, 4491.282051, 4492.820512, 4491.7...","[4492.820512, 4495.897435, 4506.666666, 4504.6...","[4380.51282, 4389.743589, 4395.897435, 4395.38...","[4050.76923, 4064.615384, 4079.487179, 4075.38...","[4246.666666, 4256.410256, 4262.564102, 4250.7...","[4195.897435, 4212.820512, 4226.153846, 4218.4...","[4195.384615, 4198.974358, 4202.051282, 4198.9...","[4228.205128, 4229.743589, 4245.128205, 4248.7...",9
3,"[4438.461538, 4427.692307, 4432.307692, 4442.5...","[4229.230769, 4222.051282, 4229.230769, 4239.4...","[4546.153846, 4538.974358, 4541.025641, 4547.1...","[4656.410256, 4647.179487, 4649.230769, 4657.9...","[4488.205128, 4475.897435, 4488.205128, 4505.6...","[4000.0, 4001.538461, 4023.589743, 4029.230769...","[4501.025641, 4495.897435, 4493.846153, 4502.5...","[4512.820512, 4507.179487, 4510.76923, 4509.74...","[4395.897435, 4390.76923, 4403.589743, 4411.28...","[4081.025641, 4077.948717, 4075.897435, 4083.0...","[4248.717948, 4243.589743, 4245.128205, 4253.8...","[4228.205128, 4210.25641, 4212.820512, 4231.28...","[4207.692307, 4203.076923, 4203.076923, 4208.2...","[4244.102564, 4234.871794, 4238.461538, 4250.7...",9
4,"[4452.820512, 4446.666666, 4449.743589, 4456.4...","[4233.846153, 4232.820512, 4230.25641, 4236.41...","[4551.282051, 4547.692307, 4548.717948, 4555.8...","[4669.743589, 4663.589743, 4665.641025, 4672.8...","[4454.358974, 4446.666666, 4448.205128, 4460.0...","[3997.435898, 3983.589744, 3992.820513, 4015.3...","[4487.692307, 4493.846153, 4489.230769, 4480.5...","[4504.615384, 4500.51282, 4498.974358, 4501.53...","[4395.384615, 4385.128205, 4382.051282, 4390.7...","[4074.358974, 4071.282051, 4074.358974, 4076.4...","[4251.282051, 4250.25641, 4251.282051, 4251.79...","[4208.717948, 4208.717948, 4216.923076, 4228.2...","[4207.179487, 4210.25641, 4212.307692, 4210.76...","[4247.179487, 4249.743589, 4242.051282, 4243.5...",0


In [3]:
# loading dataframes for each channel, pick and choose at will

# occ0 = pd.read_csv('../data/occ0Exp.csv', delimiter=',')
# occ1 = pd.read_csv('../data/occ1Exp.csv', delimiter=',')
# fefF3 = pd.read_csv('../data/fefF3Exp.csv', delimiter=',')
# fefF4 = pd.read_csv('../data/fefF4Exp.csv', delimiter=',')
# fefF7 = pd.read_csv('../data/fefF7Exp.csv', delimiter=',')
# fefF8 = pd.read_csv('../data/fefF8Exp.csv', delimiter=',')
# temT7 = pd.read_csv('../data/temT7Exp.csv', delimiter=',')
# temT8 = pd.read_csv('../data/temT8Exp.csv', delimiter=',')
# pfcAF3 = pd.read_csv('../data/pfcAF3Exp.csv', delimiter=',')
# pfcAF4 = pd.read_csv('../data/pfcAF4Exp.csv', delimiter=',')
# motFC5 = pd.read_csv('../data/motFC5Exp.csv', delimiter=',')
# motFC6 = pd.read_csv('../data/motFC6Exp.csv', delimiter=',')
# parP7 = pd.read_csv('../data/parP7Exp.csv', delimiter=',')
# parP8 = pd.read_csv('../data/parP8Exp.csv', delimiter=',')

# loading same datasets expanded with filter processing

# occ0 = pd.read_csv('../data/occ0Proc.csv', delimiter=',')
# occ1 = pd.read_csv('../data/occ1Proc.csv', delimiter=',') 
# fefF3 = pd.read_csv('../data/fefF3Proc.csv', delimiter=',')
# fefF4 = pd.read_csv('../data/fefF4Proc.csv', delimiter=',')
# fefF7 = pd.read_csv('../data/fefF7Proc.csv', delimiter=',')
# fefF8 = pd.read_csv('../data/fefF8Proc.csv', delimiter=',') 
# temT7 = pd.read_csv('../data/temT7Proc.csv', delimiter=',')
# temT8 = pd.read_csv('../data/temT8Proc.csv', delimiter=',')
# pfcAF3 = pd.read_csv('../data/pfcAF3Proc.csv', delimiter=',')
# pfcAF4 = pd.read_csv('../data/pfcAF4Proc.csv', delimiter=',')
# motFC5 = pd.read_csv('../data/motFC5Proc.csv', delimiter=',')
# motFC6 = pd.read_csv('../data/motFC6Proc.csv', delimiter=',')
# parP7 = pd.read_csv('../data/parP7Proc.csv', delimiter=',')
# parP8 = pd.read_csv('../data/parP8Proc.csv', delimiter=',')

# occ0 = pd.read_csv('../data/occ0Wide.csv', delimiter=',')
# occ1 = pd.read_csv('../data/occ1Wide.csv', delimiter=',') 
# fefF3 = pd.read_csv('../data/fefF3Wide.csv', delimiter=',')
# fefF4 = pd.read_csv('../data/fefF4Wide.csv', delimiter=',')
# fefF7 = pd.read_csv('../data/fefF7Wide.csv', delimiter=',')
# fefF8 = pd.read_csv('../data/fefF8Wide.csv', delimiter=',') 
# temT7 = pd.read_csv('../data/temT7Wide.csv', delimiter=',')
# temT8 = pd.read_csv('../data/temT8Wide.csv', delimiter=',')
# pfcAF3 = pd.read_csv('../data/pfcAF3Wide.csv', delimiter=',')
# pfcAF4 = pd.read_csv('../data/pfcAF4Wide.csv', delimiter=',')
# motFC5 = pd.read_csv('../data/motFC5Wide.csv', delimiter=',')
# motFC6 = pd.read_csv('../data/motFC6Wide.csv', delimiter=',')
# parP7 = pd.read_csv('../data/parP7Wide.csv', delimiter=',')
# parP8 = pd.read_csv('../data/parP8Wide.csv', delimiter=',')

In [ ]:
# del occ0
# del occ1
# del fefF3
# del fefF4
# del fefF7
# del fefF8
# del temT7
# del temT8
# del pfcAF3
# del pfcAF4
# del motFC5
# del motFC6
# del parP7
# del parP8

# gc.collect()

In [5]:
# loading only the dataframes from channels positively correlated with code, to save RAM
# corrcols = [occ1, fefF4, fefF8, pfcAF4, motFC6, parP8] #removed motFC5 for being a diff shape

Correlations of averaged channels, if there is a need to pick and choose to save RAM:

<sub>
code          1.000000
    
&nbsp;
    
occ0Data     -0.000385

&nbsp;
    
occ1Data      0.001626
    
&nbsp;

fefF3Data    -0.005483

&nbsp;
    
fefF4Data     0.008288

&nbsp;
    
fefF7Data    -0.004799

&nbsp;
        
fefF8Data     0.006907

&nbsp;    
    
temT7Data    -0.002657

&nbsp;    
    
temT8Data    -0.001037

&nbsp;    
    
pfcAF3Data   -0.007512

&nbsp;    
    
pfcAF4Data    0.007591

&nbsp;    
    
motFC5Data    0.003757

&nbsp;    
    
motFC6Data    0.001372

&nbsp;    
    
parP7Data    -0.003000

&nbsp;    
    
parP8Data     0.002366
</sub>

In [4]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       7.7Gi       946Mi       562Mi       6.9Gi       6.9Gi
Swap:          976Mi        11Mi       965Mi


In [7]:
# If individual columns are loaded, use this cell to process them and aggregate group them by event
# This gets rid of the time dimension though and will make the model be highly inaccurate

# dfs = [occ0, occ1, fefF3, fefF4, fefF7, fefF8, 
#        temT7, temT8, pfcAF3, pfcAF4, motFC5, motFC6, parP7, parP8]
# for i in dfs: #dfs:
# #     i.drop('Unnamed: 0.1', inplace=True, axis=1)
#     i.drop(['Unnamed: 0', 'size'], inplace=True, axis=1)
#     i['data'].astype(float, copy=False)
#     print(i.columns)
# #     i = i.groupby('event').mean()
# #     print(i.columns)

In [10]:
# non loop version of above's last line
# occ1, fefF4, fefF8, pfcAF4, motFC5, motFC6, parP8 for corrcols

# occ0 = occ0.groupby('event').mean()
# occ1 = occ1.groupby('event').mean()
# fefF3 = fefF3.groupby('event').mean()
# fefF4 = fefF4.groupby('event').mean()
# fefF7 = fefF7.groupby('event').mean()
# fefF8 = fefF8.groupby('event').mean()
# temT7 = temT7.groupby('event').mean()
# temT8 = temT8.groupby('event').mean()
# pfcAF3 = pfcAF3.groupby('event').mean()
# pfcAF4 = pfcAF4.groupby('event').mean()
# motFC5 = motFC5.groupby('event').mean() #this one has less rows than the rest, why?
# motFC6 = motFC6.groupby('event').mean()
# parP7 = parP7.groupby('event').mean()
# parP8 = parP8.groupby('event').mean()


In [8]:
## Check each channel dataframe for null vals 
# for i in corrcols:
#     print(max(i.isna().sum()))

In [58]:
## Renaming data column to specify channel name in individual channel dataframes, so when they are concatenated, the full df doesn't have multiple cols with the same names
## (use  occ1, fefF4, fefF8, pfcAF4, motFC6, parP8 for corrcols)

# occ0.rename(columns={'data': 'occ0data'}, inplace=True)
# occ1.rename(columns={'data':'occ1data', 'ICAdata':'occ1ICAdata', 'SGFdata':'occ1SGFdata'}, inplace=True)
# fefF3.rename(columns={'data': 'fefF3data'}, inplace=True)
# fefF4.rename(columns={'data':'fefF4data', 'ICAdata':'fefF4ICAdata', 'SGFdata':'fefF4SGFdata'}, inplace=True)
# fefF7.rename(columns={'data': 'fefF7data'}, inplace=True)
# fefF8.rename(columns={'data':'fefF8data', 'ICAdata':'fefF8ICAdata', 'SGFdata':'fefF8SGFdata'}, inplace=True)
# temT7.rename(columns={'data': 'temT7data'}, inplace=True)
# temT8.rename(columns={'data': 'temT8data'}, inplace=True)
# pfcAF3.rename(columns={'data': 'pfcAF3data'}, inplace=True)
# pfcAF4.rename(columns={'data':'pfcAF4data', 'ICAdata':'pfcAF4ICAdata', 'SGFdata':'pfcAF4SGFdata'}, inplace=True)
# motFC5.rename(columns={'data': 'motFC5data'}, inplace=True)
# motFC6.rename(columns={'data':'motFC6data', 'ICAdata':'motFC6ICAdata', 'SGFdata':'motFC6SGFdata'}, inplace=True)
# parP7.rename(columns={'data': 'parP7data'}, inplace=True)
# parP8.rename(columns={'data':'parP8data', 'ICAdata':'parP8ICAdata', 'SGFdata':'parP8SGFdata'}, inplace=True)


In [14]:
# occ0.columns, occ1.columns, fefF7.columns, fefF8.columns
# # dfs[0] #this wasn't changed with the above functions, it's only used for calling the original datasets as they were read in
# dfsProcessed = [occ0, occ1, fefF3, fefF4, fefF7, fefF8, 
#        temT7, temT8, pfcAF3, pfcAF4, motFC5, motFC6, parP7, parP8]

## Remaking this list included processed dataframes
# corrcolsProcessed = [occ1, fefF4, fefF8, pfcAF4, motFC6, parP8] #new vars because the original list has the unprocessed dataframes in memory it seems

In [15]:
## If using individual channel dataframes, use this cell to turn them into one df
# full = pd.concat(corrcolsProcessed, axis=1, join='inner')

In [18]:
# full.columns.duplicated()
# # Since the concat created duplicate code columns, using this trick to delete them:
# #https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
# #Although a different type of join might prevent the need for this
# full = full.loc[:,~full.columns.duplicated()].copy()
# #full = full.drop(['size', 'id' ], axis=1) # If any extraneous cols are leftover
# full.dtypes

In [9]:
# full

In [10]:
#corrMatFull = full.corr()
#corrMatFull['code']
# check = pd.read_csv('../data/occ1Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by 0.001562 
# del check 

# check = pd.read_csv('../data/fefF7Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by -0.004390.
# del check 

# check = pd.read_csv('../data/occ0Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by -0.000308 
# del check 

# check = pd.read_csv('../data/fefF8Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by 0.006184
# del check 

In [20]:
# full.plot(y=['occ0Data', 'occ1Data', 'fefF7Data', 'fefF8Data'], alpha=.7 ); 

In [85]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi        13Gi       334Mi       227Mi       1.7Gi       1.4Gi
Swap:          976Mi        45Mi       931Mi


--------------
## Baseline Model: Only O1, O2, F7, F8; No Preprocessing
## Baseline+ : Basline+Preprocessing

--------------

&nbsp;


In [183]:
## Setting up X and y, train-test split for "full"
# X = full.drop('code', axis=1)
# XExperiment = full.drop(['code', 'occ1data', 'fefF4data',
#                          'fefF8data', 'pfcAF4data', 'motFC6data', 'parP8data',
#                          'occ1SGFdata', 'fefF4SGFdata','fefF8SGFdata', 'pfcAF4SGFdata', #TODO change to pfc
#                          'motFC6SGFdata', 'parP8SGFdata'], axis=1) 
# y = full['code']
# print(X.columns, X.iloc[0], y.iloc[0], X.shape, y.shape)
# XTrain, XTest, yTrain, yTest = train_test_split(XExperiment, y)

In [5]:
# Setting up X and y, train-test split for "df"
X = df.drop('code', axis=1)
y = df['code']
print(X.iloc[0], type(y.iloc[0]), X.shape, y.shape)
XTrain, XTest, yTrain, yTest = train_test_split(X, y)

O1     [4203.076923, 4193.333333, 4194.871794, 4207.1...
O2     [4229.743589, 4216.923076, 4207.179487, 4215.3...
F3     [4538.461538, 4528.717948, 4524.615384, 4526.1...
F4     [4682.051282, 4667.179487, 4662.051282, 4669.2...
F7     [4489.230769, 4475.384615, 4474.358974, 4486.6...
F8     [3989.23077, 3983.589744, 3987.692308, 3991.79...
T7     [4497.948717, 4498.461538, 4494.871794, 4497.9...
T8     [4506.666666, 4501.025641, 4496.923076, 4496.9...
AF3    [4395.384615, 4382.564102, 4377.435897, 4387.1...
AF4    [4078.461538, 4062.564102, 4055.897435, 4065.6...
FC5    [4207.692307, 4205.641025, 4200.51282, 4194.35...
FC6    [4227.692307, 4215.384615, 4210.76923, 4221.02...
P7     [4203.076923, 4192.820512, 4194.871794, 4204.6...
P8     [4245.641025, 4236.410256, 4218.461538, 4220.0...
Name: 0, dtype: object <class 'numpy.int64'> (65034, 14) (65034,)


In [6]:
type(df.values[0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0][0])

str

In [32]:
!cowsay brains \& NN algorithms are complex!

 _____________________________________
< brains & NN algorithms are complex! >
 -------------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


In [7]:
XTrain.shape, XTest.shape, yTrain.shape, yTest.shape

((48775, 14), (16259, 14), (48775,), (16259,))

In [40]:
XTrain = tf.reshape(XTrain, (682850, 1, XTrain.shape[1], 1))
ohe = OneHotEncoder(sparse=False)
# to fix error, try this: https://www.youtube.com/watch?v=k_VAKyzggJI
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
yTrainOHE = lb.fit_transform(yTrain)
yTrainOHE = ohe.fit_transform(yTrain.to_numpy().reshape(-1,1))

# ohe0 = OneHotEncoder(sparse=False)
# yTestOHE = ohe0.fit(yTrain.to_numpy().reshape(-1,1)) # Daniel said to do this but I can't remember why

In [55]:
del model
gc.collect()

14876

In [41]:
model = Sequential()

In [42]:
model.add(Conv2D(filters=64, kernel_size=1, activation='relu', input_shape=(1,14,1)))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Conv2D(filters=64, kernel_size=1, activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Conv2D(filters=64, kernel_size=1, activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(11, activation='softmax'))

# model.add(LSTM(7, input_shape=(1,14,1), activation='tanh', 
#                recurrent_activation='sigmoid'))

In [43]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 1, 14, 64)         128       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 1, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 1, 14, 64)         4160      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 1, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 1, 14, 64)         4160      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 1, 14, 64)       

In [44]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
results = model.fit(XTrain, yTrainOHE, batch_size=20,epochs=30, verbose=1)

ValueError: Data cardinality is ambiguous:
  x sizes: 682850
  y sizes: 48775
Make sure all arrays contain the same number of samples.

Best accuracy for the unprocessed signal with all channels was about .1020

In 6/14 channels, adding SGF data was about the same

In 6/14 channels, adding SGF and dropping raw data resulted in really low accuracy and NaN loss

In 6/14 channels, ICA only had a accuracy of about .1000

6/14 channels, unproccessed+SGF+ICA, 22,059 params: NaN loss again, what's wrong with the data? woe be the futility of mine ways

6/14 channels, SGF only, 9,771 params, accuracy still around .1000

6/14 channels, raw, 9,771 params, accuracy about .1000, so the .0020 increase was only from the extra channels. Processing doesn't seem to change the accuracy at all.

----------
Using vector data rows ("wide"), I get this error:
```
UnimplementedError                        Traceback (most recent call last)
<ipython-input-59-f6344dc87bbc> in <module>
      1 model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
----> 2 results = model.fit(XTrain, yTrainOHE, batch_size=20,epochs=30, verbose=1)

...

Cast string to float is not supported
	 [[{{node sequential_4/Cast}}]] [Op:__inference_train_function_2630]
```

but the prep function seems to work fine: column is series, col[0] is list, and col[0][0] is float
```
(learn-env) :~/flatiron/cap/capstoneProject/workingNotebooks$ python dataPrepV2.py 
<class 'pandas.core.series.Series'> <class 'list'> <class 'float'>
```

-[] TODO: OHE test, validation set

### Support Vector Machine Attempt

In [109]:
svc = SVC()
svc.fit(XTrain, yTrain)

SVC()

In [116]:
svc.score(XTest, yTest)

0.10234331754720463

A quick support vector classifier ended up getting the same accuracy, 0.10234331754720463

In [197]:
XTrain.to_numpy().reshape(-1, 1).flatten(), yTrain.to_numpy().reshape(-1, 1)

(array([ 1.51991153e-05, -1.64019496e-04,  6.11943053e-05, ...,
         4.13683698e-05,  1.26545442e-04,  1.39856091e-05]),
 array([[2],
        [3],
        [0],
        ...,
        [8],
        [2],
        [0]]))

### Echo State Network Attempt

In [190]:
import reservoirpy as rpy
from reservoirpy.nodes import Reservoir, Ridge, FORCE, ESN

readout = Ridge(ridge=1e-7)
train_states = reservoir.run(XTrain.to_numpy().reshape(-1, 1), reset=True)
# train_statesy = reservoir.run(yTrain.to_numpy().reshape(-1, 1), reset=True)
readout = readout.fit(train_states, yTrain.to_numpy().reshape(-1, 1), warmup=10)
test_states = reservoir.run(XTest.to_numpy().reshape(-1, 1))
yPred = readout.run(test_states)

reservoir = Reservoir(100, lr=0.5, sr=0.9)
ridge = Ridge(ridge=1e-7)

esn_model = reservoir >> ridge

esn_model = esn_model.fit(XTrain.to_numpy().reshape(-1, 1), yTrain.to_numpy().reshape(-1, 1).to_numpy().reshape(-1, 1), warmup=10)
print(reservoir.is_initialized, readout.is_initialized, readout.fitted)

yPred = esn_model.run(XTest.to_numpy().reshape(-1, 1))


ValueError: cannot reshape array of size 292650 into shape (48775,1)

While it was nice to learn how to set one of these up, I realized after a lot of tuning the reservoirpy library doesn't seem to have good interop with pandas. Might have interesting results with 1D timestamp and 1D value arrays, but not a multidimensional dataframe

In [209]:
readout1 = Ridge(ridge=1e-7)
train_states1 = reservoir.run(full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1), reset=True)
# train_statesy = reservoir.run(yTrain.to_numpy().reshape(-1, 1), reset=True)
readout1 = readout1.fit(train_states1, full['code'].to_numpy().reshape(-1, 1), warmup=10)
test_states1 = reservoir.run((full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1)))
# yPred1 = readout.run(test_states1)

reservoir1 = Reservoir(100, lr=0.5, sr=0.9)
ridge1 = Ridge(ridge=1e-7)

esn_model1 = reservoir >> ridge

esn_model1 = esn_model1.fit(full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1), 
                             full['code'].to_numpy().reshape(-1, 1) ,warmup=10)
print(reservoir.is_initialized, readout.is_initialized, readout.fitted)

yPred1 = esn_model.run((full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1)))

Running Model-7:   0%|          | 0/1 [00:00<?, ?it/s]
Running Model-7: 608it [00:00, 6077.69it/s]           
Running Model-7: 1184it [00:00, 5977.77it/s]
Running Model-7: 1774it [00:00, 5953.20it/s]
Running Model-7: 2340it [00:00, 5861.04it/s]
Running Model-7: 2958it [00:00, 5950.73it/s]
Running Model-7: 3509it [00:00, 5809.06it/s]
Running Model-7: 4114it [00:00, 5877.67it/s]
Running Model-7: 4698it [00:00, 5864.64it/s]
Running Model-7: 5266it [00:00, 5805.29it/s]
Running Model-7: 5856it [00:01, 5832.72it/s]
Running Model-7: 6438it [00:01, 5827.26it/s]
Running Model-7: 7041it [00:01, 5886.27it/s]
Running Model-7: 7640it [00:01, 5915.91it/s]
Running Model-7: 8228it [00:01, 5904.99it/s]
Running Model-7: 8820it [00:01, 5909.08it/s]
Running Model-7: 9416it [00:01, 5922.98it/s]
Running Model-7: 10023it [00:01, 5964.50it/s]
Running Model-7: 10635it [00:01, 6009.18it/s]
Running Model-7: 11236it [00:01, 5912.81it/s]
Running Model-7: 11828it [00:02, 5818.73it/s]
Running Model-7: 12411it [00:02

Fitting node Ridge-19...


Running Model-6: 1195it [00:00, 5902.90it/s]          

True False False


Running Model-6: 65034it [00:10, 6031.98it/s]


In [220]:
set(yPred1.flatten()), (mean())

{4.494217014863551}

In [ ]:
# TODO use exploded data of one event from original CSV, predict brain activity with ESN:
# forecast, generate artificial brainwaves of looking at a digit

# TODO explore possibility of using matplotlib plots of signals as training data
# like in https://arxiv.org/abs/1511.06448

# in the future, try iterating ESNpredict -> ESNtrain -> ESNpredict -> ESNtrain -> ...
# to see how predictions change over time, maybe there would be a "core" shape that can be
# like a waveform decomposition of the number, 
